In [16]:
from google.cloud import storage


def cors_configuration(bucket_name):
    """Set a bucket's CORS policies configuration."""
    # bucket_name = "your-bucket-name"

    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    bucket.cors = [
        {
            "origin": ["*"],
            "responseHeader": [
                "Content-Type",
                "x-goog-resumable"],
            "method": ['PUT', 'POST'],
            "maxAgeSeconds": 3600
        }
    ]
    bucket.patch()

    print(f"Set CORS policies for bucket {bucket.name} is {bucket.cors}")
    return bucket
cors_configuration("opg-test-1")

DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

In [10]:
# Define the URL where Label Studio is accessible and the API key for your user account
LABEL_STUDIO_URL = 'http://localhost:8080'
# API key is available at the Account & Settings > Access Tokens page in Label Studio UI
API_KEY = '46414fdd9e8c4a843bd027f61660b9bb08114c4e'

# Import the SDK and the client module
from label_studio_sdk.client import LabelStudio

# Connect to the Label Studio API and check the connection
ls = LabelStudio(base_url=LABEL_STUDIO_URL, api_key=API_KEY)


In [ ]:

task = ls.tasks.create(
    project=3,
    data={
        "image": "gs://opg-test-1/car.jpg",  # Corrected GCS URL format
    }
)


In [ ]:
completed_annotations = ls.projects.exports.as_json(
    project.id,
    filters={"status": "Completed"}
)

In [5]:
from label_studio_sdk import LabelStudio
client = LabelStudio(
    api_key=API_KEY)

In [ ]:
from label_studio_sdk.label_interface import LabelInterface
from label_studio_sdk.label_interface.create import choices

# Define labeling interface
label_config = LabelInterface.create({
    'text': 'Text',
    'label': choices(['Positive', 'Negative'])
})

# Create a project with the specified title and labeling configuration
project = ls.projects.create(
    title='Text Classification',
    label_config=label_config
)


In [8]:
from label_studio_sdk.label_interface import LabelInterface
project = ls.projects.create(
    title="Object Detection Project",
    label_config=LabelInterface.create({
        "image": "Image",
        "bbox": LabelInterface.rectangle_labels(
            name="bbox",
            toName="image",
            labels=["Car", "Pedestrian", "Traffic Light"]
        )
    })
)


AttributeError: type object 'LabelInterface' has no attribute 'rectangle_labels'

In [6]:
client.tasks.create(
    data={"image": "car.jpg", "text": "Hello, world!"},
    project=1,
)

BaseTask(id=1, data={'image': 'car.jpg', 'text': 'Hello, world!'}, meta={}, created_at=datetime.datetime(2025, 3, 31, 7, 31, 26, 839631, tzinfo=TzInfo(UTC)), updated_at=datetime.datetime(2025, 3, 31, 7, 31, 26, 839681, tzinfo=TzInfo(UTC)), is_labeled=False, overlap=1, inner_id=2, total_annotations=0, cancelled_annotations=0, total_predictions=0, comment_count=0, unresolved_comment_count=0, last_comment_updated_at=None, project=1, updated_by=None, file_upload=None, comment_authors=[])

In [4]:
# Upload image and create task
def upload_image(project_id, image_path):
    # Upload file first
    filename = image_path.split("/")[-1]
    ls.upload_file(
        project=project_id,
        file=image_path,
        filename=filename
    )
    
    # Create task with uploaded image reference
    return ls.tasks.create(
        project=project_id,
        data={
            "image": f"file://{filename}",  # Reference uploaded file
            "meta": {"source": "local_upload"}  # Optional metadata
        }
    )

# Usage example
task = upload_image(1, "car.jpg")

AttributeError: 'LabelStudio' object has no attribute 'upload_file'